<h3>Import dependencies</h3>

In [1]:
from time import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
headerss = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)  \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
columnss = ('href', 'name', 'condition', 'date', 'bid', 'n_bid', 'rating', 'n_rat', 'seller')

<h3>Main function</h3>

In [3]:
not_parsed = []
def get_ebay_row(site):
    try:
        wrong_soup = BeautifulSoup(requests.get(site).content, 'lxml')
        correct_site = wrong_soup.find('a', {'class': 'cvip-item-card-details__view-link'})['href']
        soup = BeautifulSoup(requests.get(correct_site).content, 'lxml')
        name = soup.find('h1', {'id': 'itemTitle'}).text
        try:
            condition = soup.find('div', {'itemprop': 'itemCondition'}).text
        except:
            condition = None
        date = soup.find('span', {'id': 'bb_tlft'}).text.strip().split('\n')[0]
        bid = soup.find('span', {'id': 'convbidPrice'}).text.split('.')[0].split(' ')[0]
        n_bid = soup.find('span', {'id': 'qty-test'}).text
        try:
            rating = soup.find('span', {'class': 'ebay-review-start-rating'}).text.strip()
            n_rat = soup.find('a', {'class': 'prodreview vi-VR-prodRev'}).text.strip().split(' ')[0]
        except:
            rating, n_rat = None, None
        try:
            seller = soup.find('span', {'class': 'mbg-nw'}).text
        except:
            seller = None
        return (correct_site, name, condition, date, bid, n_bid, rating, n_rat, seller)
    except Exception as e:
        not_parsed.append(site)
        return

<h3>Collect data</h3>

In [ ]:
table = []
grandmother_url = 'https://www.ebay.com/sch/i.html?LH_Auction=1&LH_Complete=1&_from=R40&_sacat=0&_nkw=iphone+8+plus'
beggo_url = grandmother_url + '&rt=nc'
beggo_soup = BeautifulSoup(requests.get(beggo_url).content, 'lxml')
count = 0
for i in beggo_soup.find_all('a', {'class': 'vip'}):
    table.append(get_ebay_row(i['href']))
for i in range(2,34):
    count += 200
    if len(table)%100 == 0:
        print('Собрали уже {}'.format(len(table)))
        print('Сайты: {}'.format(len(not_parsed)))
    if len(shitty_sites)%100 == 0:
        print('Сайты растут: {}'.format(len(not_parsed)))
    try:
        mother_url = grandmother_url + '&_pgn=' + str(i) + '&_skc=' + str(count) + '&rt=nc'
        mother_soup = BeautifulSoup(requests.get(mother_url).content, 'lxml')
        for i in mother_soup.find_all('a', {'class': 'vip'}):
            table.append(get_ebay_row(i['href']))
    except Exception as e:
        print('На {} итерации ошибка: {}'.format(i, e))
        pass

<h3>Comvert to excel</h3>

In [ ]:
pd.DataFrame(Table, columns=columnss).to_excel(r'C:\Users\Святослав\Desktop\aleyounenenen.xlsx',index=False, header=True)